# <center> <img src="figs/LogoUFSCar.jpg" alt="Logo UFScar" width="110" align="left"/>  <br/> <center>Universidade Federal de São Carlos (UFSCar)<br/><font size="4"> Departamento de Computação, campus Sorocaba</center></font>
</p>

<font size="4"><center><b>Disciplina: Aprendizado de Máquina</b></center></font>
  
<font size="3"><center>Prof. Dr. Tiago A. Almeida</center></font>

## <center>Projeto Final</center>

**Nome**: Beatriz Rogers Tripoli Barbosa

**RA**: 792170 

**Nome**: Laura Naomi Seto

**RA**: 813210


---
### Análise exploratória

Nesta seção, deve ser feita a leitura da base de dados e todas as análises necessárias para interpretar e analisar os dados, tais como:
* Significado de cada atributo
* Medidas descritivas
* Gráficos

In [1]:
# biblioteca usada para trabalhar com vetores e matrizes
import numpy as np 

# biblioteca usada para trabalhar com dataframes e análise de dados
import pandas as pd

# bibliotecas usadas para geração de graficos
import seaborn as sns
import matplotlib.pyplot as plt

print('Bibliotecas carregadas com sucesso')

Bibliotecas carregadas com sucesso


In [3]:
# importa o arquivo e guarda em um dataframe do Pandas
df_dataset = pd.read_csv( 'data/RHP_data.csv', sep=',', index_col=None) 

print('Dados importados com sucesso!')

Dados importados com sucesso!


In [88]:
# checando valores possíveis para as colunas para análise dos atributos
# display(df_dataset["IDADE"].unique())

# null_mask = df_dataset["IDADE"].isnull()
display(df_dataset[pd.to_datetime(df_dataset['Atendimento'], errors='coerce').isna()])

C:\Users\laura\AppData\Local\Temp\ipykernel_20748\2726048506.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  display(df_dataset[pd.to_datetime(df_dataset['Atendimento'], errors='coerce').isna()])


,Id,Peso,Altura,IMC,Atendimento,DN,IDADE,Convenio,PULSOS,PA SISTOLICA,PA DIASTOLICA,PPA,B2,SOPRO,FC,HDA 1,HDA2,SEXO,MOTIVO1,MOTIVO2
64,65,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#VALUE!,NaN,NaN,NaN,NaN,NaN,Indeterminado,NaN,NaN
71,72,65.6,174,22.0,40050,36117,11.17,UR,Normais,100.0,70.0,Normal,Normal,ausente,72,Assintomático,NaN,M,5 - Parecer cardiológico,5 - Cirurgia
78,79,NaN,116,NaN,38574,35975,7.38,UR,Normais,100.0,60.0,Normal,Normal,ausente,88,Assintomático,NaN,M,6 - Suspeita de cardiopatia,6 - Dor precordial
85,86,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#VALUE!,NaN,NaN,NaN,NaN,NaN,Indeterminado,NaN,NaN
92,93,7.1,67,16.0,38185,38035,0.43,GS,NaN,NaN,NaN,Não Calculado,Normal,sistólico,100,NaN,NaN,F,6 - Suspeita de cardiopatia,6 - Sopro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17833,17834,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#VALUE!,NaN,NaN,NaN,NaN,NaN,Indeterminado,NaN,NaN
17837,17838,0.0,0,NaN,NaN,NaN,NaN,NaN,Normais,NaN,NaN,#VALUE!,Normal,ausente,110,Assintomático,NaN,Masculino,6 - Suspeita de cardiopatia,6 - Sopro
17839,17840,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#VALUE!,NaN,NaN,NaN,NaN,NaN,Indeterminado,NaN,NaN
17855,17856,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#VALUE!,NaN,NaN,NaN,NaN,NaN,Indeterminado,NaN,NaN


##### Análise dos atributos

* **Id**: Identificação do registro.

* **Peso**: Peso corporal do paciente [kg]
    * valores faltantes

* **Altura**: Altura do paciente [cm]
    * valores fora do intervalo aceitável (0 é válido?)

* **Índice de Massa Corporal** (IMC): $$\frac{Peso}{Altura^2}$$    
    * valores faltantes acarretados por: peso ou altura não registrados ou igual a 0

* **Atendimento**: Data de atendimento [DD/MM/AA]
    * valores inválidos

* **Data de Nascimento** (DN): Data de nascimento [DD/MM/AA]
    * valores faltantes
    * valores inválidos

* **Idade**: Idade no momento do atendimento [anos]
    * valores não numéricos ('#VALUE!') acarretados por: atendimento e/ou data de nascimento inválidos 
    * valores fora do intervalo aceitável (idades negativas)
    * valores faltantes acarretados por: data de nascimento não registrado
    * valores aparentemente válidos mas atendimento e/ou data de nascimento inválidos

* **Convênio**: Tipo de plano de saúde utilizado para o atendimento

* **Pulsos**: Avaliação da presença e qualidade dos pulsos arteriais [normais, outro, amplos, femorais diminuidos, diminuídos]
    * adotar formas canônicas para os valores possíveis

* **PA Sistólica**: Valor mais alto registrado durante o batimento cardíaco [mmHg]

* **PA Diastólica**: Valor mais baixo registrado entre os batimentos cardíacos [mmHg]

* **Pressão de Pulso Arterial** (PPA): [não calculado, normal, pre-hipertensão pas, has-2 pas, pre-hipertensão pad, has-1 pas, has-2 pad, has-1 pad]
    * valor inválido ('#VALUE!')

* **B2**: Segundo som cardíaco [normal, desdob fixo, outro, hiperfonética, única]

* **Sopro**: Sopro cardíaco [sistólico, ausente, contínuo, diastólico, 'sistolico e diastólico']
    * adotar formas canônicas para os valores possíveis 

* **Frequência Cardíaca** (FC): Número de batimentos cardíacos por minuto [bpm]

* **História da Doença Atual 1** (HDA 1): registro de um problema de saúde do paciente

* **História da Doença Atual 2** (HDA2): registro de um problema de saúde do paciente

* **Sexo**: Gênero biológico do paciente [masculino, feminino e indeterminado]
    * adotar formas canônicas para os valores possíveis

* **Motivo 1**: Principal motivo para o atendimento

* **Motivo 2**: Motivo secundário para o atendimento

##### Abordagens
**Valores faltantes e inválidos** - analisar porcentagem de valores faltantes e natureza dos dados (numérico, categórico, temporais)
- exclusão de linha (poucos faltantes)
- exclusão de coluna (muitos faltantes)
- substituição por média, mediana (melhor quando há outliers) ou moda (dados categóricos)
- substituição por interpolação (adequada para séries temporais)

**Duplicatas** (todos os valores de atributos iguais e mesma classificação) 
- exclusão de linha

**Inconsistências** (todos os valores de atributos iguais e classificados diferentemente)
- exclusão das linhas


#### Exploração dos dados com estatística descritiva

Resumo quantitativo das principais características de um conjunto de dados:
* Frequência
* Localização ou tendência central
* Dispersão ou espalhamento
* Distribuição ou formato

Os três últimos são utilizados geralmente para valores numéricos.

Para dados univariados: 
* Medidas de localidade: moda, média, mediana, média truncada, quartis, percentil (boxplot)
* Medidas de espalhamento: intervalo, variância e descio padrão
* Medidas de distribuição: obliquidade ou assimetria e curtose (histograma e gráfico pizza)

Para dados multivariados:
* Medidas de localidade: calculado para cada atributo separadamente
* Medidas de espalhamento: calculado apra cada atributo separadamente + covariância e correlação (matriz de covariância ou correlação)

---
### Pré-processamento

Nesta seção, as funções da etapa de pré-processamento dos dados devem ser implementadas e aplicadas (se necessário).

---
### Experimento

Nesta seção, o experimento deve ser conduzido, utilizando os protocolos experimentais padrões e testando diferentes modelos.

---
### Análise dos Resultados

Nesta seção, os resultados devem ser exibidos através de tabelas e gráficos, comparados e profundamente analisados.